In [31]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [32]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [44]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('data.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

C:\Users\Dung\AppData\Local\Temp\ipykernel_17240\681518283.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('data.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
C:\Users\Dung\AppData\Local\Temp\ipykernel_17240\681518283.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('data.csv', index_col="date", infer_datetime_format=True, parse_dates=True)


,fng_value
date,
2024-11-26,79
2024-11-25,82
2024-11-24,80
2024-11-23,93
2024-11-22,94


In [72]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('bitcoin_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

C:\Users\Dung\AppData\Local\Temp\ipykernel_17240\413883731.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df2 = pd.read_csv('bitcoin_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']


Date
31/08/2023    25931.47266
31/08/2024    58969.89844
31/10/2023    34667.78125
31/10/2024    70215.18750
31/12/2023    42265.18750
Name: Close, dtype: float64

In [45]:
print(df.dtypes)

fng_value    int64
dtype: object


In [48]:
common_dates = df.index.intersection(df2.index)
print("Common dates:", common_dates)

Common dates: Index([], dtype='object')


In [51]:
df.index = pd.to_datetime(df.index, dayfirst=True)
df2.index = pd.to_datetime(df2.index, dayfirst=True)

In [52]:
print(df.dtypes)

fng_value    int64
dtype: object


In [53]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2023-01-05,29,16836.73633
2023-01-04,29,16863.23828
2023-01-03,26,16679.85742
2023-01-02,27,16688.47070
2023-01-01,26,16625.08008


In [54]:
df.head()

,fng_value,Close
2024-11-26,79,94412.47656
2024-11-24,80,98013.82031
2024-11-23,93,97777.28125
2024-11-22,94,98997.66406
2024-11-21,82,98504.72656


In [55]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [56]:
# Predict Closing Prices using a 1 day window of previous closing prices
window_size = 1

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [57]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

In [58]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [59]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(y)

y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [60]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [62]:
# Define the LSTM RNN model.
model = Sequential()

number_units = 50
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [63]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [64]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             10400     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             20200     
                                                                 
 dropout_1 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 5

In [65]:
# Train the model
model.fit(X_train, y_train, epochs=15, shuffle=False, batch_size=1, verbose=1)

Epoch 1/15
484/484 [==============================] - 20s 21ms/step - loss: 0.0443
Epoch 2/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0154
Epoch 3/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0140
Epoch 4/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0116
Epoch 5/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0082
Epoch 6/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0056
Epoch 7/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0037
Epoch 8/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0035
Epoch 9/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0026
Epoch 10/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0020
Epoch 11/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0018
Epoch 12/15
484/484 [==============================] - 10s 21ms/step - loss: 0.0023
E

## Model Performance

In [66]:
# Evaluate the model
model.evaluate(X_test, y_test)

7/7 [==============================] - 3s 17ms/step - loss: 8.0446e-04


0.0008044629939831793

In [67]:
# Make some predictions
predicted = model.predict(X_test)

7/7 [==============================] - 2s 9ms/step


In [68]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [69]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices):]) 
stocks.head()

,Real,Predicted
2023-07-27,29319.24609,29361.273438
2023-07-26,29210.68945,29340.621094
2023-07-25,29354.97266,29281.138672
2023-07-24,29227.39063,29360.208984
2023-07-23,29176.91602,29290.285156


In [70]:
stocks.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)